In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction 

This notebook offers demonstrations of different methods of querying data that is stored in BigQuery from within the JupyterLab interface of your Vertex AI Workbench instance.

Confused ? Let's define each of these terms in case they're new to you

 * **BigQuery** BigQuery is a fully managed, AI-ready data platform that helps you manage and analyze your data with built-in features like machine learning, search, geospatial analysis, and business intelligence. BigQuery's serverless architecture lets you use languages like SQL and Python to answer your organization's biggest questions with zero infrastructure management.
 
 * **JupyterLab**  JupyterLab is a highly extensible, feature-rich notebook authoring application and editing environment, and is a part of Project Jupyter, a large umbrella project centered around the goal of providing tools (and standards) for interactive computing with computational notebooks.
 
 * **Vertex AI Workbench instance** Vertex AI Workbench instances are Jupyter notebook-based development environments on Google Cloud for the entire data science workflow. Vertex AI Workbench instances are prepackaged with JupyterLab. Vertex AI Workbench instances have integrations and features can make it easier to access your data, process data faster, schedule notebook runs, and more.
 
For most of these methods, we will use a **Python JupyterLab Kernel**. In the Jupyter architecture, kernels are separate processes started by the server that run your code in different programming languages and environments.  We will use the IPython Jupyter Kernel that comes prepackaged with Vertex AI Workbench Instances. This will allow us to execute Python code in this notebook interactively.

Reference: 

https://cloud.google.com/bigquery/docs/introduction

https://jupyterlab.readthedocs.io/en/latest/user/interface.html

https://cloud.google.com/vertex-ai/docs/workbench/instances/introduction

https://jupyterlab.readthedocs.io/en/latest/user/documents_kernels.html#kernel-backed-documents

https://ipython.readthedocs.io/en/stable/index.html

# Method 1 : Using the BigQuery Pane in Vertex AI workbnench

This is Method 1 of querying data that is stored in BigQuery from within the JupyterLab interface of your Vertex AI Workbench instance.

BigQuery is integrated into the the JupyterLab interface of your Vertex AI Workbench instance.

Through this integration you can use a BigQuery pane that lists available projects and datasets. 

Let's use this BigQuery pane now to explore some BigQuery public datasets. 

 * select project "bigquery-public-data"
   * select datatset "google_trends"
     * select table "international_top_terms"
       * click "preview" to see a sample of rows from this table
       * click "query table" to open the Stand-alone BigQuery query editor opens as a separate tab in JupyterLab.

Reference : 

https://cloud.google.com/vertex-ai/docs/workbench/instances/bigquery#browse_resources

https://cloud.google.com/vertex-ai/docs/workbench/instances/bigquery#stand-alone



# Method 2 : Using the %%bigquery magic command

This is Method 2 of querying data that is stored in BigQuery from within the JupyterLab interface of your Vertex AI Workbench instance.

For this method, it is required to use the IPython Jupyter Kernel that comes prepackaged with Vertex AI Workbench Instances. Please select & connect to it now.

The IPython Jupyter Kernel has a feature called "cell magics" which are handy commands built into the IPython kernel that make it easy to perform particular tasks.

BigQuery has developed "cell magics" to make it easy to execute SQL queries. There are two BigQuery "cell magics" 

1. %%bigquery

  *  Behind the scenes, the %%bigquery magic command uses the BigQuery client library for Python to run the given query
  *  Then convert the results to a pandas DataFrame 
  *  Then display results.

2. %%bigquery my_pandas_data_frame

  *  Behind the scenes, the %%bigquery magic command uses the BigQuery client library for Python to run the given query
  *  Then convert the results to a pandas DataFrame 
  *  Then save the pandas DataFrame to the variable my_pandas_data_frame
  
See exampe of these two "cell magics" below

Reference:

https://cloud.google.com/vertex-ai/docs/workbench/instances/bigquery#query_data_by_using_the_bigquery_magic_command

https://ipython.readthedocs.io/en/stable/interactive/magics.html#cell-magics

https://ipython.readthedocs.io/en/stable/interactive/magics.html#line-magics

https://cloud.google.com/python/docs/reference/bigquery/latest/magics


In [ ]:
%%bigquery
SELECT
  country_code,
  country_name,
  COUNT(DISTINCT region_code) AS num_regions
FROM
  `bigquery-public-data.google_trends.international_top_terms`
WHERE
  refresh_date = DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
GROUP BY
  country_code,
  country_name
ORDER BY
  num_regions DESC;

In [ ]:
%%bigquery regions_by_country
SELECT
  country_code,
  country_name,
  COUNT(DISTINCT region_code) AS num_regions
FROM
  `bigquery-public-data.google_trends.international_top_terms`
WHERE
  refresh_date = DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
GROUP BY
  country_code, country_name
ORDER BY
  num_regions DESC;

In [ ]:
regions_by_country.head()

# Method 3 : Using the BigQuery python client library 

This is Method 3 of querying data that is stored in BigQuery from within the JupyterLab interface of your Vertex AI Workbench instance.

For this method, it is required to use the IPython Jupyter Kernel that comes prepackaged with Vertex AI Workbench Instances. Please select & connect to it now.

As per the example below, using the BigQuery client library involves: 

1. importing the library
2. initialing a BigQuery client object
3. defining your SQL query
4. getting the BigQuery client object to execute SQL and return a pandas Dataframe
5. with the returned pandas Dataframe you can use a number of Python data analysis, data wrangling, and visualization libraries



Reference :

https://cloud.google.com/vertex-ai/docs/workbench/instances/bigquery#query_data_by_using_the_client_library_directly




In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

In [ ]:
sql = """
WITH
  TopTermsByDate AS (
    SELECT DISTINCT refresh_date AS date, term
    FROM `bigquery-public-data.google_trends.top_terms`
  ),
  DistinctDates AS (
    SELECT DISTINCT date
    FROM TopTermsByDate
  )
SELECT
  DATE_DIFF(Dates2.date, Date1Terms.date, DAY)
    AS days_apart,
  COUNT(DISTINCT (Dates2.date || Date1Terms.date))
    AS num_date_pairs,
  COUNT(Date1Terms.term) AS num_date1_terms,
  SUM(IF(Date2Terms.term IS NOT NULL, 1, 0))
    AS overlap_terms,
  SAFE_DIVIDE(
    SUM(IF(Date2Terms.term IS NOT NULL, 1, 0)),
    COUNT(Date1Terms.term)
    ) AS pct_overlap_terms
FROM
  TopTermsByDate AS Date1Terms
CROSS JOIN
  DistinctDates AS Dates2
LEFT JOIN
  TopTermsByDate AS Date2Terms
  ON
    Dates2.date = Date2Terms.date
    AND Date1Terms.term = Date2Terms.term
WHERE
  Date1Terms.date <= Dates2.date
GROUP BY
  days_apart

ORDER BY
  days_apart;
"""
pct_overlap_terms_by_days_apart = client.query(sql).to_dataframe()

pct_overlap_terms_by_days_apart.head()

# Method 4 : Using the BigQuery integration into Vertex AI workbench

This is Method 4 of querying data that is stored in BigQuery from within the JupyterLab interface of your Vertex AI Workbench instance.

The In-cell BigQuery query editor is a cell type that you can use within your notebook files.

Select the BigQuery icon on a given cell to launc the In-cell BigQuery query editor. 

Reference : 

https://cloud.google.com/vertex-ai/docs/workbench/instances/bigquery#in-cell



# Method 5 : Use BigQuery DataFrames aka "BigFrames" to super charge pandas workloads

This is Method 6 of querying data that is stored in BigQuery from within the JupyterLab interface of your Vertex AI Workbench instance.

For this method, it is required to use the IPython Jupyter Kernel that comes prepackaged with Vertex AI Workbench Instances. Please select & connect to it now.

BigQuery DataFrames provides a Python library **bigframes.pandas** which provides a pandas-compatible API for analytics. 

Through this library you take advantage of BigQuery data processing by using familiar pandas python APIs. BigQuery DataFrames provides a Pythonic DataFrame powered by the BigQuery engine, and it implements the pandas  APIs by pushing the processing down to BigQuery through SQL conversion. This lets you use BigQuery to explore and process terabytes of data with Python APIs.


Reference : 

https://cloud.google.com/bigquery/docs/bigquery-dataframes-introduction

https://cloud.google.com/bigquery/docs/use-bigquery-dataframes

https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks

https://cloud.google.com/deep-learning-vm/docs/introduction#pre-installed_packages

https://cloud.google.com/vertex-ai/docs/workbench/instances/add-environment



# Method 6 : Use BigQuery DataFrames aka "BigFrames" to super charge scikit learn workloads

This is Method 7 of querying data that is stored in BigQuery from within the JupyterLab interface of your Vertex AI Workbench instance.

For this method, it is required to use the IPython Jupyter Kernel that comes prepackaged with Vertex AI Workbench Instances. Please select & connect to it now.

BigQuery DataFrames provides a library **bigframes.ml** which provides a scikit-learn-like API for machine learning (ML). 

Through this library you take advantage of BigQuery data processing by using familiar Python scikit-learn-like APIs. BigQuery DataFrames provides a Pythonic DataFrame powered by the BigQuery engine, and it implements the scikit-learn APIs by pushing the processing down to BigQuery through SQL conversion (specifically BigQuery ML). This lets you use BigQuery to explore and process terabytes of data, and also train machine learning (ML) models, all with Python APIs.

Reference : 

https://cloud.google.com/bigquery/docs/bigquery-dataframes-introduction

https://cloud.google.com/bigquery/docs/use-bigquery-dataframes

https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks

https://cloud.google.com/bigquery/docs/bqml-introduction

https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/getting_started/ml_fundamentals_bq_dataframes.ipynb

https://cloud.google.com/deep-learning-vm/docs/introduction#pre-installed_packages

https://cloud.google.com/vertex-ai/docs/workbench/instances/add-environment

In [ ]:
!pip install bigframes

In [ ]:
import bigframes.pandas as bpd

In [ ]:
df = bpd.read_gbq("bigquery-public-data.ml_datasets.penguins")
df = df.dropna()

# BigQuery DataFrames creates a default numbered index, which we can give a name
df.index.name = "penguin_id"

In [ ]:
# Isolate input features and output variable into DataFrames
X = df[['island', 'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'sex', 'species']]
y = df[['body_mass_g']]

In [ ]:
from bigframes.ml.model_selection import train_test_split

# This will split X and y into test and training sets, with 20% of the rows in the test set,
# and the rest in the training set
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2)

# Show the shape of the data after the split
print(f"""X_train shape: {X_train.shape}
X_test shape: {X_test.shape}
y_train shape: {y_train.shape}
y_test shape: {y_test.shape}""")

In [ ]:
from bigframes.ml.preprocessing import StandardScaler

# StandardScaler will only work on numeric columns
numeric_columns = ["culmen_length_mm", "culmen_depth_mm", "flipper_length_mm"]

scaler = StandardScaler()
scaler.fit(X_train[numeric_columns])

# Now, standardscaler should transform the numbers to have mean of zero
# and standard deviation of one:
scaler.transform(X_train[numeric_columns])

In [ ]:
scaler.transform(X_test[numeric_columns])


In [ ]:
from bigframes.ml.compose import ColumnTransformer
from bigframes.ml.preprocessing import OneHotEncoder

# Create an aggregate transform that applies StandardScaler to the numeric columns,
# and OneHotEncoder to the string columns
preproc = ColumnTransformer([
    ("scale", StandardScaler(), ["culmen_length_mm", "culmen_depth_mm", "flipper_length_mm"]),
    ("encode", OneHotEncoder(), ["species", "sex", "island"])])

# Now we can fit all columns of the training data
preproc.fit(X_train)

processed_X_train = preproc.transform(X_train)
processed_X_test = preproc.transform(X_test)

# View the processed training data
processed_X_train

In [ ]:
from bigframes.ml.linear_model import LinearRegression

linreg = LinearRegression()

# Learn from the training data how to predict output y
linreg.fit(processed_X_train, y_train)

# Predict y for the test data
predicted_y_test = linreg.predict(processed_X_test)

# View predictions
predicted_y_test